<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the relevant packages
from IPython.display import Image
import os
import numpy as np
import pandas as pd
import ast
import time 
import random
from datetime import datetime
from sklearn import ensemble, metrics, linear_model
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.svm import SVR

In [ ]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset


 97% 220M/228M [00:02<00:00, 110MB/s]
100% 228M/228M [00:02<00:00, 115MB/s]


In [ ]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = '/content/the-movies-dataset.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [ ]:
Metadata = pd.read_csv('movies_metadata.csv')
Nmovies = Metadata['id'].shape[0]
print(Nmovies)
Metadata.head(5)

45466


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
RatingsDataFrame = pd.read_csv('ratings.csv')
Nusers = RatingsDataFrame['userId'].nunique()
print(Nusers)
RatingsDataFrame.tail(30)

270896


,userId,movieId,rating,timestamp
26024259,270896,41566,3.5,1257034128
26024260,270896,44191,5.0,1257031546
26024261,270896,44195,4.0,1257034427
26024262,270896,45499,3.0,1257034131
26024263,270896,45722,3.5,1257031715
26024264,270896,46578,4.5,1257034037
26024265,270896,48385,4.0,1257031939
26024266,270896,48394,5.0,1257031861
26024267,270896,48516,4.5,1257031526
26024268,270896,48774,5.0,1257032040


In [ ]:
MovieFeatures = pd.DataFrame()
# keep the numerical columns untouched for now
MovieFeatures[0] = Metadata['id']
MovieFeatures[1] = Metadata['budget']
MovieFeatures[2] = Metadata['popularity']
MovieFeatures[3] = Metadata['revenue']
MovieFeatures[4] = Metadata['runtime']
MovieFeatures[5] = Metadata['vote_average']
MovieFeatures[6] = Metadata['vote_count']

In [ ]:
# extracting the year from the release date feature
release_date = np.array(Metadata['release_date'])
for i in range(0,Nmovies):
    if (type(release_date[i])==str and \
        len(release_date[i]) == 10):
        year = datetime.strptime(release_date[i], '%Y-%m-%d')
        year = year.year
        release_date[i] = year
    else:
        release_date[i] = float('nan')

for i in range(1,31): 
    if (type(release_date[-i])==str and \
        len(release_date[-i]) == 10): 
        year = datetime.strptime(release_date[-i], '%Y-%m-%d') 
        year = year.year 
        release_date[-i] = year 
    else: 
        release_date[-i] = float('nan') 

MovieFeatures[7] = release_date

In [ ]:
Metadata['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:
# this function extracts words out of dictionaries and formats them
def word_extractor(row_of_words):
    words_joined = []
    if (type(row_of_words)!=str or type(ast.literal_eval(row_of_words))!=list):
        words_joined = ['']
    else:
    # extract words from the dictionaries
        word_list = ast.literal_eval(row_of_words)
        for w in range(0,len(word_list)):
            word_list[w] = word_list[w]['name']
            word_list[w] = word_list[w].replace(" ","")
        words_joined.append(' '.join(word_list))    
    return words_joined

# applying the word_extractor function to dict. features
genres = []
for m in range(0,Nmovies):
    genres.append(word_extractor(Metadata['genres'][m]))

genres = np.array(genres)
genres = genres.ravel()
print(genres[0])

Animation Comedy Family


In [ ]:
# one-hot encoding binary features
le = LabelEncoder()
lb = LabelBinarizer()
original_language = le.fit_transform(Metadata['original_language'].fillna('0'))
original_language = lb.fit_transform(original_language)
for i in range(8,8+original_language.shape[1]):
    MovieFeatures[i] = original_language[:,i-8]
    
# finally, vectorizing the features.
count_vectorizer = CountVectorizer()
tfid = TfidfVectorizer(stop_words={'english','french','spanish','german'},\
                  max_features=200)

genres = count_vectorizer.fit_transform(genres)
original_title = tfid.fit_transform(Metadata['original_title'])
overview = tfid.fit_transform(Metadata['overview'].values.astype('U'))
tagline = tfid.fit_transform(Metadata['tagline'].values.astype('U'))

In [ ]:
genres[2].todense()
genres[20].todense()

matrix([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [ ]:
print("finally, vectorizing the features... \n")
count_vectorizer = CountVectorizer()
tfid = TfidfVectorizer(stop_words={'english','french','spanish','german'},\
                  max_features=200)

original_title = tfid.fit_transform(Metadata['original_title'])
overview = tfid.fit_transform(Metadata['overview'].values.astype('U'))
tagline = tfid.fit_transform(Metadata['tagline'].values.astype('U'))

# this function records the processed data in the MovieFeatures DataFrame
def record_new_data(new_data):
  size = MovieFeatures.shape[1]
  for i in range(size,size+new_data.shape[1]):
    MovieFeatures[i] = new_data.toarray()[:,i-size]

record_new_data(genres)
record_new_data(original_title)
record_new_data(overview)
record_new_data(tagline)

genres = genres.toarray()
print("Getting rid of NaN values... \n")
Metadata_mean = Metadata.mean(skipna=True,numeric_only=True)
MovieFeatures[3] = MovieFeatures[3].fillna(Metadata_mean['revenue'])
MovieFeatures[4] = MovieFeatures[4].fillna(Metadata_mean['runtime'])
MovieFeatures[5] = MovieFeatures[5].fillna(Metadata_mean['vote_average'])
MovieFeatures[6] = MovieFeatures[6].fillna(Metadata_mean['vote_count'])
MovieFeatures = MovieFeatures.fillna('0')


finally, vectorizing the features... 

Getting rid of NaN values... 



In [ ]:
MovieFeatures.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732
0,862,30000000,21.9469,373554033.0,81.0,7.7,5415.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,8844,65000000,17.0155,262797249.0,104.0,6.9,2413.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,15602,0,11.7129,0.0,101.0,6.5,92.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,31357,16000000,3.85949,81452156.0,127.0,6.1,34.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.238996,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.362289,0.0,0.0
4,11862,0,8.38752,76578911.0,106.0,5.7,173.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.196315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.283225,0.0,0.000000,0.0,0.0,0.0,0.0,0.27732,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [ ]:
def user_dataframe(active_user):
  user_df = RatingsDataFrame.groupby('userId').get_group(active_user)
  user_df = PCA_df.merge(user_df,left_on=0,right_on='movieId')
  return user_df

In [ ]:
# Step 1: this function splits a user dataframe into training and test data
def test_split(active_user):
  percentage = 0.85
  user_df = user_dataframe(active_user)
  X = user_df.iloc[:,1:3]
  y = user_df.iloc[:,5]
  x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.15)
  return x_train, x_test, y_train, y_test

In [ ]:
# this function finds the best hyperparameters for an svr user model
def svr_tuning(active_user):
  parameters = {'C':[0.1, 1, 10],'epsilon':[0.1,0.2,0.5],'gamma':['auto','scale']}
  x_train, x_test, y_train, y_test = test_split(active_user)
  svr = SVR(gamma='scale')
  svr = GridSearchCV(svr, parameters, random_state=0)
  search = svr.fit(x_train, y_train)
  return search.best_params_

# this function finds the best hyperparameters for an gbr user model
def GBR_tuning(active_user):
  parameters = {'n_estimators':[100,300,500],'learning_rate':[0.001,0.01,0.1,1],\
    'loss':['ls','lad','huber','quantile']}
  x_train, x_test, y_train, y_test = test_split(active_user)
  GBR = ensemble.GradientBoostingRegressor()
  clf = GridSearchCV(GBR, parameters)
  search = clf.fit(x_train, y_train)
  return search.best_params_

In [ ]:
# this function creates an svr model for an active user
def training(active_user):
  x_train, x_test, y_train, y_test = test_split(active_user)
  svr = SVR(gamma='auto', epsilon=0.2, C=0.1)
  LR = linear_model.LinearRegression()
  GBR = ensemble.GradientBoostingRegressor(learning_rate=0.001,loss='ls',\
    n_estimators=100)
  svr.fit(x_train, y_train)
  predicted = svr.predict(x_test)
  model_rmse = np.sqrt(metrics.mean_squared_error(y_test,predicted))
  return svr, [x_test, y_test], model_rmse

In [ ]:
# this function returns the N largest elements of a list
def Nmaxelements(list1, N): 
    final_list = []  
    for i in range(0, N):  
        max1 = 0          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j];                  
        list1.remove(max1); 
        final_list.append(max1)          
    return final_list 

# this function creates an svr model for an active user
def recommendations(active_user,n_recom):
  svr, testdata, model_rmse = training(active_user)  
  recommend = svr.predict(pca_features)
  recommend_max = Nmaxelements(recommend.tolist(),n_recom)
  suggestions = []
  genres_array = np.zeros(20)
  for i in range(0,len(recommend)): 
     if recommend[i] in recommend_max:
       suggestions.append(Metadata['original_title'][i])
       suggestions.append(word_extractor(Metadata['genres'][i]))
       genres_array = genres_array + genres[i]
  return genres_array, suggestions, model_rmse

In [ ]:
# this function computes a (N movies watched)-weighted accuracy avg for N users
def accuracy(N):
  counter = 0
  R2 = 0
  error_rsme = 0
  for i in range(1,N):
    active_user = random.randint(1,Nusers)
    user_df = user_dataframe(active_user)
    Nmovies_rated = user_df.shape[0]
    if (Nmovies_rated > 10):
      svr, testdata, model_rmse = training(active_user)
      predicted = svr.predict(testdata[0])
      rmse = np.sqrt(metrics.mean_squared_error(testdata[1],predicted))
      error_rsme = error_rsme + Nmovies_rated*rmse
      test_score = svr.score(testdata[0], testdata[1])
      R2 = R2 + Nmovies_rated*test_score
      counter = counter + Nmovies_rated
  error_rsme = error_rsme/counter
  R2 = R2/counter
  return error_rsme, R2

In [ ]:
# this function computes a diversity index for an active user
def diversity(active_user,n_recom):
  genres_counter = 0
  Ngenres = 20
  user_df = user_dataframe(active_user)
  Nmovies_rated = user_df.shape[0]
  if (Nmovies_rated > 1):
    genres_array, suggestions, model_rmse = recommendations(active_user,n_recom)
    for i in range(0,genres_array.size):
      if (genres_array[i] > 0):
        genres_counter = genres_counter + 1
  genres_counter = genres_counter/Ngenres
  return genres_counter

# this function computes a diversity avg index for N users
def diversity_avg(N,n_recom):
  diversity_total = 0
  for user in range(1,N):
    diversity_total = diversity_total + diversity(user,n_recom)
  diversity_total = diversity_total/N
  return diversity_total

In [ ]:
# this function measures the algorithmic runtime and accuracy for N user models
def model_performance(N):
  t1 = time.clock()
  error_rsme, R2 = accuracy(N)
  t2 = time.clock()
  t  = t2-t1
  return error_rsme, R2, t

In [ ]:
recommendations(2,5)

